In [1]:
import pandas as pd
from datetime import datetime
from functions_load_and_transform import thisWeek_DF, lastWeek_DF, thisWeek, lastWeek, team_list, scoringDF, player_list

In [8]:
# Load Data
schedule = pd.read_csv("data/schedule.csv", delimiter=";")
schedule["Date"] = pd.to_datetime(schedule["Date"], format="%Y.%m.%d %H:%M")

thisDay = datetime.today().strftime("%Y-%m-%d") # for live
thisDay = '2024-09-23'

In [22]:
for day in range(15, 31):
    thisDay = f'2024-09-{day}'
    thisWeek = list(schedule.loc[schedule["Date"]>=thisDay, "Week"])[0]
    print(thisDay, thisWeek)


2024-09-15 2
2024-09-16 2
2024-09-17 3
2024-09-18 3
2024-09-19 3
2024-09-20 3
2024-09-21 3
2024-09-22 3
2024-09-23 3
2024-09-24 4
2024-09-25 4
2024-09-26 4
2024-09-27 4
2024-09-28 4
2024-09-29 4
2024-09-30 4
